In [82]:
import pandas as pd
import numpy as np
import json
import csv
from elasticsearch import helpers, Elasticsearch
import requests
import re
import os

In [83]:
df = pd.read_csv("listings_2.csv")

In [84]:
df = df[[                "id",
                "listing_url",
                "scrape_id",
                "last_scraped",
                "source",
                "name",
                "description",
                "neighborhood_overview",
                "picture_url",
                "host_id",
                "host_url",
                "host_name",
                "host_since",
                "host_location",
                "host_about",
                "host_response_time",
                "host_response_rate",
                "host_acceptance_rate",
                "host_is_superhost",
                "host_thumbnail_url",
                "host_picture_url",
                "host_neighbourhood",
                "host_listings_count",
                "host_total_listings_count",
                "host_verifications",
                "host_has_profile_pic",                
                "host_identity_verified",  
                "neighbourhood",                
                "neighbourhood_cleansed",
                "neighbourhood_group_cleansed",
                "latitude",
                "longitude",
                "property_type",                
                "room_type",
                "accommodates",                
                "bathrooms",
                "bathrooms_text",
                "bedrooms",
                "beds",
                "amenities",
                "price",
                "minimum_nights",
                "maximum_nights",
                "minimum_minimum_nights",
                "maximum_minimum_nights",
                "minimum_maximum_nights",
                "maximum_maximum_nights",
                "minimum_nights_avg_ntm",                
                "calendar_updated",
                "has_availability",
                "availability_30",
                "availability_60",
                "availability_90",
                "availability_365",
                "calendar_last_scraped",
                "number_of_reviews",
                "number_of_reviews_ltm",
                "number_of_reviews_l30d",
                "first_review",
                "last_review",
                "review_scores_rating",
                "review_scores_accuracy",                
                "review_scores_cleanliness",                
                "review_scores_checkin",                
                "review_scores_communication",                
                "review_scores_location",                
                "review_scores_value",                
                "license",                
                "instant_bookable",                
                "calculated_host_listings_count",                
                "calculated_host_listings_count_entire_homes",                
                "calculated_host_listings_count_private_rooms",                
                "calculated_host_listings_count_shared_rooms",                
                "reviews_per_month"
]]

In [85]:
reg = "\$([0-9]*)(\.|\,){1,1}.*"
df["price"] = df["price"].apply(lambda x: re.search(reg, str(x)).group(1))
df["price"] = df["price"].astype("float")

In [86]:
df.fillna(0)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5396,https://www.airbnb.com/rooms/5396,20221210143007,2022-12-11,city scrape,Explore the heart of old Paris,"Cozy, well-appointed and graciously designed s...","You are within walking distance to the Louvre,...",https://a0.muscache.com/pictures/52413/f9bf76f...,7903,...,4.82,4.95,4.55,7510402838018,f,1,1,0,0,1.98
1,7397,https://www.airbnb.com/rooms/7397,20221210143007,2022-12-11,city scrape,MARAIS - 2ROOMS APT - 2/4 PEOPLE,"VERY CONVENIENT, WITH THE BEST LOCATION !<br /...",0,https://a0.muscache.com/pictures/67928287/330b...,2626,...,4.88,4.93,4.72,7510400829623,f,2,2,0,0,2.26
2,7964,https://www.airbnb.com/rooms/7964,20221210143007,2022-12-11,previous scrape,Large & sunny flat with balcony !,Very large & nice apartment all for you! <br /...,0,https://a0.muscache.com/pictures/4471349/6fb3d...,22155,...,5.00,5.00,5.00,7510903576564,f,1,1,0,0,0.04
3,9359,https://www.airbnb.com/rooms/9359,20221210143007,2022-12-11,city scrape,"Cozy, Central Paris: WALK or VELIB EVERYWHERE !",Location! Location! Location! Just bring your ...,0,https://a0.muscache.com/pictures/c2965945-061f...,28422,...,0.00,0.00,0.00,"Available with a mobility lease only (""bail mo...",f,1,1,0,0,0.00
4,81870,https://www.airbnb.com/rooms/81870,20221210143007,2022-12-11,previous scrape,Saint Germain Musee d'orsay,<b>The space</b><br />This beautiful apartment...,0,https://a0.muscache.com/pictures/558458/3c1263...,152242,...,5.00,5.00,4.00,0,f,78,78,0,0,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55099,777293887700658247,https://www.airbnb.com/rooms/777293887700658247,20221210143007,2022-12-11,city scrape,Aborigine,The flat is decorated in a pure and design way...,Very quiet area.<br />Four times a week there ...,https://a0.muscache.com/pictures/miso/Hosting-...,490045822,...,0.00,0.00,0.00,7511107644867,f,1,1,0,0,0.00
55100,778194850167838333,https://www.airbnb.com/rooms/778194850167838333,20221210143007,2022-12-11,city scrape,Gratte-ciel - bien située - proche Tour Eiffel,"In the heart of the 15th arrondissement, this ...",The 15th arrondissement is a pleasant and very...,https://a0.muscache.com/pictures/prohost-api/H...,490713158,...,0.00,0.00,0.00,7511507662375,f,1,1,0,0,0.00
55101,778209219542952721,https://www.airbnb.com/rooms/778209219542952721,20221210143007,2022-12-11,city scrape,Charming apartment 2 P - Malesherbe,This pretty studio specially designed for love...,This studio is located in the 8th arrondisseme...,https://a0.muscache.com/pictures/prohost-api/H...,374553401,...,0.00,0.00,0.00,7510807648613,f,91,91,0,0,0.00
55102,777311545564408798,https://www.airbnb.com/rooms/777311545564408798,20221210143007,2022-12-10,city scrape,Appartement Guisarde,Profitez d'un logement élégant et central. Le ...,0,https://a0.muscache.com/pictures/miso/Hosting-...,325898345,...,0.00,0.00,0.00,7510807670783,f,4,4,0,0,0.00


In [13]:
df.head(10)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5396,https://www.airbnb.com/rooms/5396,20221210143007,2022-12-11,city scrape,Explore the heart of old Paris,"Cozy, well-appointed and graciously designed s...","You are within walking distance to the Louvre,...",https://a0.muscache.com/pictures/52413/f9bf76f...,7903,...,4.82,4.95,4.55,7510402838018,f,1,1,0,0,1.98
1,7397,https://www.airbnb.com/rooms/7397,20221210143007,2022-12-11,city scrape,MARAIS - 2ROOMS APT - 2/4 PEOPLE,"VERY CONVENIENT, WITH THE BEST LOCATION !<br /...",NaN,https://a0.muscache.com/pictures/67928287/330b...,2626,...,4.88,4.93,4.72,7510400829623,f,2,2,0,0,2.26
2,7964,https://www.airbnb.com/rooms/7964,20221210143007,2022-12-11,previous scrape,Large & sunny flat with balcony !,Very large & nice apartment all for you! <br /...,NaN,https://a0.muscache.com/pictures/4471349/6fb3d...,22155,...,5.00,5.00,5.00,7510903576564,f,1,1,0,0,0.04
3,9359,https://www.airbnb.com/rooms/9359,20221210143007,2022-12-11,city scrape,"Cozy, Central Paris: WALK or VELIB EVERYWHERE !",Location! Location! Location! Just bring your ...,NaN,https://a0.muscache.com/pictures/c2965945-061f...,28422,...,NaN,NaN,NaN,"Available with a mobility lease only (""bail mo...",f,1,1,0,0,NaN
4,81870,https://www.airbnb.com/rooms/81870,20221210143007,2022-12-11,previous scrape,Saint Germain Musee d'orsay,<b>The space</b><br />This beautiful apartment...,NaN,https://a0.muscache.com/pictures/558458/3c1263...,152242,...,5.00,5.00,4.00,NaN,f,78,78,0,0,0.01
5,9952,https://www.airbnb.com/rooms/9952,20221210143007,2022-12-11,city scrape,Paris petit coin douillet,"Je suis une dame retraitée, qui propose un agr...","Vibrant neighborhood, full of bars, cafés, fre...",https://a0.muscache.com/pictures/ae822d16-74d2...,33534,...,4.84,4.88,4.91,7511101582862,f,1,1,0,0,0.36
6,84820,https://www.airbnb.com/rooms/84820,20221210143007,2022-12-11,city scrape,Bird's nest in Montmartre: Sun/view all over P...,"Apartment, two rooms, very calm and sunny.<br ...","So many bars, restaurants, shops, 5mn from Sac...",https://a0.muscache.com/pictures/581023/86105d...,464019,...,4.88,4.90,4.77,7511801291867,f,1,1,0,0,0.83
7,245715,https://www.airbnb.com/rooms/245715,20221210143007,2022-12-11,previous scrape,"Apart Hotel, Paris Champs Elysee",<b>The space</b><br />@ Home Suite Home Paris ...,NaN,https://a0.muscache.com/pictures/23451178/47bc...,1275649,...,4.18,4.91,4.55,NaN,f,2,2,0,0,0.32
8,10586,https://www.airbnb.com/rooms/10586,20221210143007,2022-12-12,city scrape,Studio 7 Montmartre,Furnished rental BAIL MOBILITE <br />(lease...,"Montmartre, the neighborhood surrounding the s...",https://a0.muscache.com/pictures/445716/5811c9...,37107,...,4.98,4.60,4.67,"Available with a mobility lease only (""bail mo...",f,4,3,1,0,0.31
9,246647,https://www.airbnb.com/rooms/246647,20221210143007,2022-12-11,city scrape,charming duplex,<b>The space</b><br />incredibile duplex - lof...,"very central, 15 mn walk from the marais, 8 mn...",https://a0.muscache.com/pictures/miso/Hosting-...,1293609,...,5.00,4.83,4.33,7511206011082,f,1,1,0,0,0.06


In [68]:
df.fillna(0).to_csv("short.csv", sep=',')

In [15]:
mappings = {
    "settings" : {
        "number_of_shards" : 1,
        "default_pipeline" : "timestamp_pipeline"
    },
    "mappings" : {   
            "properties" : {
                "id" : {"type" : "integer"},
                "listing_url" : {"type" : "keyword"},
                "scrape_id" : {"type" : "keyword"},
                "last_scraped" : {"type" : "keyword"},
                "source" : {"type" : "keyword"},
                "name" : {"type" : "keyword"},
                "description" : {"type" : "keyword"},
                "neighborhood_overview" : {"type" : "keyword"},
                "picture_url" : {"type" : "keyword"},
                "host_id" : {"type" : "integer"},
                "host_url" : {"type" : "keyword"},
                "host_name" : {"type" : "keyword"},
                "host_since" : {"type" : "keyword"},
                "host_location" : {"type" : "keyword"},
                "host_about" : {"type" : "keyword"},
                "host_response_time" : {"type" : "keyword"},
                "host_response_rate" : {"type" : "keyword"},
                "host_acceptance_rate" : {"type" : "keyword"},
                "host_is_superhost" : {"type" : "keyword"},
                "host_thumbnail_url" : {"type" : "keyword"},
                "host_picture_url" : {"type" : "keyword"},
                "host_neighbourhood" : {"type" : "keyword"},
                "host_listings_count" : {"type" : "integer"},
                "host_total_listings_count" : {"type" : "integer"},
                "host_verifications" : {"type" : "keyword"},
                "host_has_profile_pic" : {"type" : "keyword"},                
                "host_identity_verified" : {"type" : "keyword"},  
                "neighbourhood" : {"type" : "keyword"},                
                "neighbourhood_cleansed" : {"type" : "keyword"},
                "neighbourhood_group_cleansed" : {"type" : "keyword"},
                "latitude" : {"type" : "float"},
                "longitude" : {"type" : "float"},
                "property_type" : {"type" : "keyword"},                
                "room_type" : {"type" : "keyword"},
                "accommodates" : {"type" : "integer"},                
                "bathrooms" : {"type" : "integer"},
                "bathrooms_text" : {"type" : "keyword"},
                "bedrooms" : {"type" : "integer"},
                "beds": {"type" : "integer"},
                "amenities": {"type" : "keyword"},
                "price" : {"type" : "float"},
                "minimum_nights" : {"type" : "integer"},
                "maximum_nights" : {"type" : "integer"},
                "minimum_minimum_nights" : {"type" : "integer"},
                "maximum_minimum_nights" : {"type" : "integer"},
                "minimum_maximum_nights" : {"type" : "integer"},
                "maximum_maximum_nights" : {"type" : "integer"},
                "minimum_nights_avg_ntm" : {"type" : "float"},                
                "calendar_updated" : {"type" : "keyword"},
                "has_availability" : {"type" : "keyword"},
                "availability_30" : {"type" : "integer"},
                "availability_60" : {"type" : "integer"},
                "availability_90" : {"type" : "integer"},
                "availability_365" : {"type" : "integer"},
                "calendar_last_scraped" : {"type" : "keyword"},
                "number_of_reviews" : {"type" : "integer"},
                "number_of_reviews_ltm" : {"type" : "integer"},
                "number_of_reviews_l30d" : {"type" : "integer"},
                "first_review" : {"type" : "keyword"},
                "last_review" : {"type" : "keyword"},
                "review_scores_rating" : {"type" : "float"},
                "review_scores_accuracy" : {"type" : "float"},                
                "review_scores_cleanliness" : {"type" : "float"},                
                "review_scores_checkin" : {"type" : "float"},                
                "review_scores_communication" : {"type" : "float"},                
                "review_scores_location" : {"type" : "float"},                
                "review_scores_value" : {"type" : "float"},                
                "license" : {"type" : "keyword"},                
                "instant_bookable" : {"type" : "keyword"},                
                "calculated_host_listings_count" : {"type" : "integer"},                
                "calculated_host_listings_count_entire_homes" : {"type" : "integer"},                
                "calculated_host_listings_count_private_rooms" : {"type" : "integer"},                
                "calculated_host_listings_count_shared_rooms" : {"type" : "integer"},                
                "reviews_per_month" : {"type" : "float"},                
                "location" : {"type": "geo_point"},
                "@timestamp" : {"type" : "date"},
            }
    }
}

In [16]:
INDEX_NAME = 'test'

In [19]:
es = Elasticsearch(
    "https://test-b08acd.es.uksouth.azure.elastic-cloud.com:9243",
    basic_auth=("elastic", "VOFvk91wj06rn5SkllRm6ElJ")
)

In [21]:
es.indices.delete(index=INDEX_NAME)

ObjectApiResponse({'acknowledged': True})

In [87]:
es.indices.create(index=INDEX_NAME, body=mappings)

/tmp/ipykernel_15186/2332734024.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=INDEX_NAME, body=mappings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test'})

In [23]:
timestamp_pipeline_setting = {
    "description" : "set a timestamp field for all documents",
    "processors" : [
        {
            "set": {
                "field": "@timestamp",
                "value": "{{_ingest.timestamp}}"
            }
        }
    ]
}

es.ingest.put_pipeline(id="timestamp_pipeline", body=timestamp_pipeline_setting)

/tmp/ipykernel_15186/1623491344.py:13: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.ingest.put_pipeline(id="timestamp_pipeline", body=timestamp_pipeline_setting)


ObjectApiResponse({'acknowledged': True})

In [88]:
df["location"] = df["latitude"].astype(str) +", "+df["longitude"].astype(str)
df.pop(df.columns[0])
#df.head(10).fillna(0).to_csv("short.csv", sep=',')
df.fillna(0).to_csv("short.csv", sep=',')

In [89]:
df = pd.read_csv("short.csv")
json_str = df.to_json(orient='records')

json_records = json.loads(json_str)

es = Elasticsearch(
    "https://test-b08acd.es.uksouth.azure.elastic-cloud.com:9243",
    basic_auth=("elastic", "VOFvk91wj06rn5SkllRm6ElJ"),
)
index_name = 'test'
doctype = '_doc'
action_list = []
for row in json_records:
    record ={
        '_op_type': 'index',
        '_index': index_name,
        '_source': row
    }
    action_list.append(record)
helpers.bulk(es, action_list, raise_on_error=False)

/tmp/ipykernel_15186/4064914736.py:1: DtypeWarning: Columns (2,22,35,40,44,45,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("short.csv")


(55104,
 [{'index': {'_index': 'test',
    '_id': '4RZqrYYBLVdEJHa9aatL',
    'status': 400,
    'error': {'type': 'mapper_parsing_exception',
     'reason': "failed to parse field [price] of type [float] in document with id '4RZqrYYBLVdEJHa9aatL'. Preview of field's value: '2022-12-10'",
     'caused_by': {'type': 'number_format_exception',
      'reason': 'For input string: "2022-12-10"'}}}}])